In [24]:
import pandas as pd 
import numpy as np
from bertopic import BERTopic

#load data 
import pandas as pd 
 
df = pd.read_csv("./data_fixed.csv", engine='python',sep='^')
#df = df.astype(str)
docs= []
valueList = df.values.tolist()
for value in valueList:
    docs.append(value[0])

#todo change to hash table

In [25]:
from transformers.pipelines import pipeline
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords 
pt_stopwords = stopwords.words('portuguese')
pt_stopwords.extend(['caput','art','inciso','artigo','i','ii','iii','iv','incisos','167'])

# we add this to remove stopwords
#vectorizer_model = CountVectorizer(strip_accents='ascii',ngram_range=(1, 2), stop_words=pt_stopwords)
vectorizer_model = CountVectorizer(stop_words=pt_stopwords)
#embedding_model = pipeline("feature-extraction", model="ulysses-camara/legal-bert-pt-br")



sentence_model = SentenceTransformer("ulysses-camara/legal-bert-pt-br")
model = BERTopic(embedding_model=sentence_model,
                 vectorizer_model=vectorizer_model,
                 calculate_probabilities=True,
                 language='portuguese')

# model = BERTopic(
#     vectorizer_model=vectorizer_model,
#     #language='portuguese', calculate_probabilities=True,
#     embedding_model=sentence_model,
#     verbose=True
# )


topics, probabilities = model.fit_transform(docs)

In [26]:
model.get_topic_info().head(5)

,Topic,Count,Name
0,-1,1026,-1_lei_federal_nacional_sobre
1,0,185,0_organização_ministério_administrativa_seguintes
2,1,115,1_deste_aplica_disposto_desta
3,2,84,2_distrito_estados_municípios_federal
4,3,64,3_cargo_cargos_emprego_função


In [27]:
model.get_topic(0)

[('organização', 0.02342360496987051),
 ('ministério', 0.0223456615902352),
 ('administrativa', 0.02123159525347466),
 ('seguintes', 0.020630987484251184),
 ('lei', 0.02031602848102163),
 ('controle', 0.02011984663471591),
 ('público', 0.019492163316409218),
 ('princípios', 0.01861632468189262),
 ('poder', 0.017204199266000343),
 ('normas', 0.016767909076801285)]

In [28]:
model.visualize_barchart()

In [29]:
from scipy.cluster import hierarchy as sch
# Hierarchical topics
linkage_function = lambda x: sch.linkage(x, 'single', optimal_ordering=True)
hierarchical_topics = model.hierarchical_topics(docs, linkage_function=linkage_function)
model.visualize_hierarchy(hierarchical_topics=hierarchical_topics,orientation='bottom')

100%|██████████| 43/43 [00:00<00:00, 45.83it/s]


In [30]:
from sentence_transformers import SentenceTransformer

#sentence_model = SentenceTransformer("ulysses-camara/legal-bert-pt-br")
embeddings = sentence_model.encode(docs, show_progress_bar=False)


model.visualize_hierarchical_documents(docs, hierarchical_topics, embeddings=embeddings)


In [31]:
T = model.get_document_info(docs)
display(T)

,Document,Topic,Name,Top_n_words,Probability,Representative_document
0,a soberania;,20,20_polícia_ferroviária_rodoviária_anistia,polícia - ferroviária - rodoviária - anistia -...,1.000000,False
1,a cidadania;,-1,-1_lei_federal_nacional_sobre,lei - federal - nacional - sobre - estado - fo...,0.689377,False
2,a dignidade da pessoa humana;,31,31_políticos_direitos_rádio_televisão,políticos - direitos - rádio - televisão - ind...,0.488535,False
3,os valores sociais do trabalho e da livre inic...,-1,-1_lei_federal_nacional_sobre,lei - federal - nacional - sobre - estado - fo...,0.308213,False
4,o pluralismo político.,6,6_cultura_culturais_tecnologia_apoiará,cultura - culturais - tecnologia - apoiará - p...,1.000000,False
...,...,...,...,...,...,...
2508,As leis previstas no inciso III do § 1º do art...,1,1_deste_aplica_disposto_desta,deste - aplica - disposto - desta - constituiç...,1.000000,True
2509,"Na hipótese de insuficiência de desempenho, a ...",-1,-1_lei_federal_nacional_sobre,lei - federal - nacional - sobre - estado - fo...,0.186501,False
2510,"Os benefícios pagos, a qualquer título, pelo ó...",40,40_natureza_benefícios_ativos_recursos,natureza - benefícios - ativos - recursos - pe...,1.000000,False
2511,Com o objetivo de assegurar recursos para o pa...,40,40_natureza_benefícios_ativos_recursos,natureza - benefícios - ativos - recursos - pe...,1.000000,False


In [34]:
tree =model.get_topic_tree(hierarchical_topics,tight_layout=True)
f = open('topics.txt','w',encoding='utf-8')
f.write(tree)

f.close()

In [33]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import re
mname = "cristian-popa/bart-tl-ng"
tokenizer = AutoTokenizer.from_pretrained(mname)
seq2seq_model = AutoModelForSeq2SeqLM.from_pretrained(mname)
f = open('finalized_data.csv','w',encoding='utf-8')

for index, row in T.iterrows():
    value =  re.sub(r'(-*[1-9]|[1-9][0-9]|[1-9][0-9][0-9]|[1-9][0-9][0-9][0-9])', "", row['Name'])
    value = value[1:]
    value = value.replace("_", " ")
    #input = value
    print(value)
    f.write(row['Document'] + "$" + str(row['Topic']) +  "$" + value +"\n"  )
    # enc = tokenizer(input, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
    # outputs = model.generate(
    #     input_ids=enc.input_ids,
    #     attention_mask=enc.attention_mask,
    #     max_length=15,
    #     min_length=1,
    #     do_sample=False,
    #     num_beams=25,
    #     length_penalty=1.0,
    #     repetition_penalty=1.5
    # )

    # decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(decoded) # windows live messengere
    
f.close()



 polícia ferroviária rodoviária anistia
lei federal nacional sobre
políticos direitos rádio televisão
lei federal nacional sobre
cultura culturais tecnologia apoiará
lei federal nacional sobre
lei federal nacional sobre
lei federal nacional sobre
deficiência integração portadoras portador
lei federal nacional sobre
lei federal nacional sobre
lei federal nacional sobre
 organização ministério administrativa seguintes
 polícia ferroviária rodoviária anistia
políticos direitos rádio televisão
 polícia ferroviária rodoviária anistia
 polícia ferroviária rodoviária anistia
lei federal nacional sobre
lei federal nacional sobre
lei federal nacional sobre
prisão ninguém julgado sentença
lei federal nacional sobre
 polícia ferroviária rodoviária anistia
 organização ministério administrativa seguintes
lei federal nacional sobre
lei federal nacional sobre
prisão ninguém julgado sentença
prisão ninguém julgado sentença
lei federal nacional sobre
dano danos indenização culpa
lei federal nacional s